**Visualization - 1. Basic Plots (Line, Bar, Scatter)**

Data visualization is essential for exploring data, communicating findings, and identifying patterns that might not be obvious from raw numbers. This guide covers three fundamental plot types used to show trends (line plots), comparisons (bar plots), and relationships (scatter plots).

Based on [`piplist.txt`](./README.md) output, you should have `matplotlib` and `seaborn`, Python's standard libraries for creating professional visualizations.

**Step 1: Generate Sample Data for Basic Plots**

We'll create a single dummy dataset containing a date column (for a line plot), a categorical column (for a bar plot), and two numerical columns with some correlation (for a scatter plot).

In a new Excel cell, enter `=PY` and paste the following code, then press **Ctrl+Enter**:

In [ ]:
# Generate dummy data for Basic Plots
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import date, timedelta

fake = Faker()

num_records = 500
start_date = date(2023, 1, 1)
dates = pd.date_range(start=start_date, periods=num_records, freq='D')

# Data for Line Plot (Value over Time)
time_series_value = np.linspace(50, 200, num_records) + np.random.normal(0, 10, num_records) # Trend + Noise
# Add some seasonality (e.g., simple weekly)
day_of_week = dates.dayofweek.values
time_series_value += np.sin(day_of_week * (2 * np.pi / 7)) * 15


# Data for Bar Plot (Value/Count per Category)
categories = ['Category A', 'Category B', 'Category C', 'Category D', 'Category E']
category_data = random.choices(categories, weights=[0.25, 0.2, 0.15, 0.2, 0.2], k=num_records)


# Data for Scatter Plot (Two correlated numerical variables)
# Create variable 1
variable1 = np.random.uniform(10, 100, num_records)
# Create variable 2 that is somewhat correlated with variable 1
variable2 = variable1 * random.uniform(0.8, 1.2) + np.random.normal(0, 15, num_records) # Add some noise


data = {
    'Date': dates,
    'TimeSeriesValue': time_series_value.round(2),
    'Category': category_data,
    'Numerical_X': variable1.round(2),
    'Numerical_Y': variable2.round(2)
}

df_basic_plots = pd.DataFrame(data)

# Add some missing values
missing_indices_ts = random.sample(range(num_records), int(num_records * 0.03))
df_basic_plots.loc[missing_indices_ts, 'TimeSeriesValue'] = np.nan

missing_indices_cat = random.sample(range(num_records), int(num_records * 0.02))
df_basic_plots.loc[missing_indices_cat, 'Category'] = np.nan # Use np.nan for pandas missing value


df_basic_plots # Output the DataFrame

**Explanation:**

*   This code generates a DataFrame `df_basic_plots` with five columns suitable for the three basic plot types:
    *   `Date` and `TimeSeriesValue` for a line plot.
    *   `Category` for a bar plot (we'll count occurrences).
    *   `Numerical_X` and `Numerical_Y` for a scatter plot, designed to have a positive correlation.
*   Missing values are introduced.
*   The result, `df_basic_plots`, will be spilled into your Excel sheet. Let's assume this data is placed in a range or Table named `BasicPlotData`.

**Step 2: Create and Style Basic Plots**

Now, we'll load this dummy data and create a line plot, a bar plot, and a scatter plot, applying the specified style guidelines.

In a **new** Excel cell, enter `=PY` and paste the following code. Replace `"BasicPlotData"` with the actual name of the Excel range/Table where your dummy data is. Press **Ctrl+Enter**.

In [ ]:
# Create and style basic plots (Line, Bar, Scatter)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data from Excel
# IMPORTANT: Replace "BasicPlotData" with the actual name of your Excel range or Table
df = xl("BasicPlotData[#All]", headers=True)

# Ensure 'Date' is a datetime column and numerical columns are numeric
df['Date'] = pd.to_datetime(df['Date'])
df['TimeSeriesValue'] = pd.to_numeric(df['TimeSeriesValue'], errors='coerce')
df['Numerical_X'] = pd.to_numeric(df['Numerical_X'], errors='coerce')
df['Numerical_Y'] = pd.to_numeric(df['Numerical_Y'], errors='coerce')
df['Category'] = df['Category'].astype(str).replace('nan', np.nan) # Handle NaN in Category


# --- Apply Custom Style Guidelines ---
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.edgecolor'] = '#1a1a24'
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.grid'] = False # Turn off default grid
sns.set_theme(style="whitegrid") # Use a seaborn theme base, then apply customs


# --- 1. Line Plot (Time Series Trend) ---
# Drop NaNs for plotting the line
df_line = df.dropna(subset=['Date', 'TimeSeriesValue'])

fig1, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(df_line['Date'], df_line['TimeSeriesValue'], label='Value', color='#188ce5', linewidth=1.5) # Blue

ax1.set_title('Time Series Value Trend', fontsize=14, color='#1a1a24')
ax1.set_xlabel('Date', fontsize=12, color='#1a1a24')
ax1.set_ylabel('Value', fontsize=12, color='#1a1a24')
sns.despine(ax=ax1, top=True, right=True)
ax1.grid(False)
fig1.autofmt_xdate() # Auto-format date labels


plt.tight_layout()


# --- 2. Bar Plot (Category Counts) ---
# Calculate counts per category, including NaN as 'Missing'
category_counts = df['Category'].value_counts(dropna=False).reset_index()
category_counts.columns = ['Category', 'Count']
# Replace NaN index value with 'Missing' if needed (value_counts with dropna=False uses NaN index)
category_counts['Category'] = category_counts['Category'].replace({np.nan: 'Missing'})

fig2, ax2 = plt.subplots(figsize=(8, 6))
# Using a categorical color palette from the guidelines
colors = ['#ffe600', '#188ce5', '#2db757', '#ff6d00', '#750e5c', '#ff4136'] # Yellow, Blue, Green, Orange, Purple, Salmon

# Use seaborn barplot on the calculated counts
sns.barplot(x='Count', y='Category', hue='Category', legend=False,  data=category_counts, ax=ax2, palette=colors[:len(category_counts)], orient='h')

ax2.set_title('Count per Category', fontsize=14, color='#1a1a24')
ax2.set_xlabel('Count', fontsize=12, color='#1a1a24')
ax2.set_ylabel('Category', fontsize=12, color='#1a1a24')
sns.despine(ax=ax2, top=True, right=True)
ax2.grid(False)

# Add data labels (counts)
for container in ax2.containers:
    ax2.bar_label(container, fmt='{:,.0f}', color='#1a1a24')


plt.tight_layout()


# --- 3. Scatter Plot (Numerical Relationship) ---
# Drop NaNs for plotting the scatter plot
df_scatter = df.dropna(subset=['Numerical_X', 'Numerical_Y'])

fig3, ax3 = plt.subplots(figsize=(8, 6))
sns.scatterplot(x='Numerical_X', y='Numerical_Y', data=df_scatter, ax=ax3, color='#750e5c', alpha=0.7, s=50) # Purple, slightly transparent dots

ax3.set_title('Numerical_Y vs. Numerical_X', fontsize=14, color='#1a1a24')
ax3.set_xlabel('Numerical_X', fontsize=12, color='#1a1a24')
ax3.set_ylabel('Numerical_Y', fontsize=12, color='#1a1a24')
sns.despine(ax=ax3, top=True, right=True)
ax3.grid(False)

plt.tight_layout()


# Output results
# Return a dictionary containing the plot figures
output = {
    'TimeSeries_Line_Plot': fig1,
    'Category_Counts_Bar_Plot': fig2,
    'Numerical_Relationship_Scatter_Plot': fig3,
}

output # Output the dictionary

**Explanation:**

*   We load the dummy data using `xl()`. **Remember to replace `"BasicPlotData"`**.
*   We ensure relevant columns have appropriate data types, handling missing values (`NaN`) gracefully using `errors='coerce'` for numerics and explicitly converting 'nan' strings from Excel load to `np.nan` for categoricals.
*   **Style Guidelines:** We set Matplotlib's `rcParams` and use `seaborn.set_theme()` to apply the specified style globally before creating the plots.
*   **Line Plot:**
    *   We filter out rows with missing dates or `TimeSeriesValue` for a clean line.
    *   `ax.plot()` is used to draw the line, mapping 'Date' to the x-axis and 'TimeSeriesValue' to the y-axis.
    *   Applied blue color (`#188ce5`), title, labels, removed top/right spines, and turned off the grid. Auto-formats date labels.
*   **Bar Plot:**
    *   We calculate frequency counts for the `Category` column using `value_counts(dropna=False)` to include missing values.
    *   `seaborn.barplot()` is used, plotting the 'Count' for each 'Category'. `orient='h'` makes it a horizontal bar plot which is often better for category labels.
    *   Applied colors from the guideline palette, title, labels, removed top/right spines, turned off the grid, and added data labels showing the counts on the bars.
*   **Scatter Plot:**
    *   We filter out rows with missing values in `Numerical_X` or `Numerical_Y`.
    *   `seaborn.scatterplot()` is used to plot individual points, mapping `Numerical_X` to the x-axis and `Numerical_Y` to the y-axis.
    *   Applied purple color (`#750e5c`) with some transparency (`alpha`), title, labels, removed top/right spines, and turned off the grid.
*   We return a dictionary containing the three Matplotlib figure objects.

**Viewing the Output:**

*   Click the Python cell, then click the Python icon/button next to the formula bar.
*   For each plot figure object ('TimeSeries_Line_Plot', 'Category_Counts_Bar_Plot', 'Numerical_Relationship_Scatter_Plot'), select "Picture in Cell" > "Create Reference" to see the plots spilled into your worksheet.

These basic plots form the foundation for understanding data distributions, trends, and relationships in your data analysis workflow.

**Further Analysis:**

Here are some advanced visualization techniques you could apply to this dataset:

1. **Interactive Visualizations:**
   - Implement interactive plots using Plotly
   - Add hover tooltips with detailed information
   - Create linked views where selecting data in one plot highlights related data in others

2. **Advanced Plot Customization:**
   - Create dual-axis plots combining different plot types
   - Implement custom color palettes based on corporate branding
   - Add annotations and reference lines for key events or thresholds

3. **Dynamic Visualizations:**
   - Create animated plots showing changes over time
   - Implement sliding window visualizations
   - Build real-time updating plots for streaming data

4. **Layout and Composition:**
   - Create multi-panel plots with different aspects of the data
   - Implement hierarchical visualizations
   - Design responsive layouts that adapt to different display sizes

5. **Statistical Visualization Enhancement:**
   - Add confidence intervals to trend lines
   - Implement kernel density estimation overlays
   - Show statistical summaries alongside raw data

The next topic in the series is [Visualization - Distribution Plots (Histogram, Box Plot, KDE)](./06-Visualization_02-Distribution%20Plots%20(Histogram,%20Box%20Plot,%20KDE).md), which explores techniques for visualizing data distributions and statistical properties.